In [1]:
# Imports required libarie
import pandas as pd
import numpy as np
import spacy
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF

In [2]:
# Load spacy 
nlp = spacy.load("en_core_web_sm")

C:\Users\gaikw\AppData\Roaming\Python\Python38\site-packages\spacy\util.py:693: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) requires spaCy >=3.0.0,<3.1.0 and is incompatible with the current version (3.0.0rc5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [7]:
sc_india_1999 = pd.read_csv('Supreme_Court_cases_India_tokens_above_1999_final.csv')
if 'Unnamed: 0' in sc_india_1999.columns:
    sc_india_1999.drop(['Unnamed: 0'], axis = 1, inplace = True)
sc_india_1999.head()

,Document Id,Tokens,Corpus
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...


In [8]:
sc_india_1999[sc_india_1999.Corpus.isnull()]

,Document Id,Tokens,Corpus
11697,1331690,[],NaN
11698,1001110,[],NaN
11736,1407971,[],NaN
11737,1322423,[],NaN
11738,1699303,[],NaN
11837,642450,[],NaN
11838,1820430,[],NaN
11839,95614585,[],NaN
12121,513704,[],NaN
12291,772112,[],NaN


In [9]:
sc_india_1999.dropna(inplace=True)

In [10]:
sc_india_1999.iloc[19347]

Document Id                                             43180661
Tokens         ['banumathi', 'jleave', 'prefer', 'judgment', ...
Corpus         banumathi jleave prefer judgment date pass hig...
Name: 19382, dtype: object

In [73]:
def nmf_topics(corpus):
    
    max_df = 0.2
    min_df=0.1
    topics = 20
    features = 750000
    top_words = 100
    
    tfidf_vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df, max_features = features)
    
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    nmf = NMF(solver='mu',max_iter=10000,n_components = topics,random_state = 42, alpha = 0.1, l1_ratio = 0.5).fit(tfidf_matrix)
    print(nmf.components_)
    
    topicdict = {}
    for idx, topic in enumerate(nmf.components_):
        topicdict[idx]=" ".join([tfidf_features[i] for i in topic.argsort()[:-top_words -1:-1]])
    return tfidf_matrix, nmf, topicdict,tfidf_features


In [74]:
tfidf_matrix, nmf_test, nmfword,feature_names = nmf_topics(sc_india_1999['Corpus'])

[[1.38367990e-01 1.45463039e-17 4.82476949e-01 ... 1.52976499e-03
  4.85817588e-02 5.33319273e-02]
 [2.36874719e-01 2.64692200e-01 0.00000000e+00 ... 4.01696933e-01
  2.02142208e-01 3.38236298e-01]
 [0.00000000e+00 1.27169487e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 1.78914809e-01 ... 1.00875345e-06
  0.00000000e+00 2.62097941e-02]
 [0.00000000e+00 2.53924775e-01 0.00000000e+00 ... 1.06825326e-01
  0.00000000e+00 0.00000000e+00]
 [4.76618868e-02 4.73332538e-02 0.00000000e+00 ... 1.39620270e-02
  3.76697924e-02 0.00000000e+00]]


In [75]:
#nmf_test=pd.DataFrame(nmf_test)
#nmf_test.to_pickle('nmfmodelfull.pickle')
nmfword=pd.DataFrame(nmfword,index=[1])
nmfword.to_pickle('nmfdictfull.pickle')
nmfword=pd.read_pickle('nmfdictfull.pickle')
nmfword

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,pw deceased incident death occurrence convicti...,committee schedule review list regulation comm...,plaintiff decree title deed share execute owne...,tax commissioner sale assess revenue entry pay...,compensation deceased acquire loss owner value...,complaint magistrate complainant fir quash all...,industrial corporation employ financial contra...,employee scale enquiry regulation regular empl...,agreement contract supply construction execute...,imprisonment conviction undergo fine convict p...,appointment list regular chief category admini...,transfer family title share deed competent ses...,deposit interim week stay money withdraw defau...,bank regulation recovery transaction financial...,notification acquire declaration value commiss...,sale value deed sell execute purchase transact...,decree execution execute share substantial fam...,injury medical death intention defence inciden...,scheme development sanction policy acquire app...,premise business control revision let owner fa...


In [76]:




def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()



In [77]:
#print topic words    
print("\nTopics in NMF model:")

print_top_words(nmf_test, feature_names,25)


Topics in NMF model:
Topic #0:
pw deceased incident death occurrence conviction die pm dead injury station fir convict brother session hospital version crime acquit presence tell lodge guilty investigate away
Topic #1:
committee schedule review list regulation commissioner institution affidavit delay policy society certificate office director chief representation development category entry limitation applicant legislature test enquiry function
Topic #2:
plaintiff decree title deed share execute owner sale limitation declaration die death joint family substantial bar agreement plead execution purchase revenue pray prayer brother money
Topic #3:
tax commissioner sale assess revenue entry payable business purchase liability return demand collect definition total loss schedule legislature industrial chapter meaning explanation define expression value
Topic #4:
compensation deceased acquire loss owner value liability payable development national death policy future road increase total redu

In [78]:
#tfidf_vectorizer=pd.DataFrame(tfidf_vectorizer)
#tfidf_vectorizer.to_pickle('tfidf_vect.pickle')
tfidf_vectorizer

TfidfVectorizer(max_df=0.75, max_features=750000, min_df=0.1)

In [79]:
out = nmf_test.transform(tfidf_matrix)

import operator
topics = []
for item in out:
    max_ix, max_val = max(enumerate(item), key = operator.itemgetter(1))
    topics.append(max_ix)
    
sc_india_1999["topicnumber"] = pd.Series(topics, index=cleaned.index)

NameError: name 'cleaned' is not defined

In [80]:
# cleaned.topicnumber.value_counts()

In [81]:
cleaned

NameError: name 'cleaned' is not defined

In [ ]:
sc_india_1999['Corpus']['year']=sc_india.Year[:100]

In [ ]:
cleaned

In [ ]:
year_df = cleaned[['year','topicnumber']]
year_df['count']=1
year_df = year_df.groupby(['year', 'topicnumber']).count().reset_index()

In [ ]:
year_df['yearcount']=0
year_df.groupby('year')['count'].sum()

In [ ]:
year_df

In [ ]:
cleaned.to_pickle('cleaned.pickle')
year_df.to_pickle('year_df.pickle')